In [1]:
import mysql.connector
import pypyodbc as odbc;
import pandas as pd;


In [2]:
db = mysql.connector.connect(host="localhost",user="mysql",password="mysql",database="Hospital")

In [3]:
mycursor = db.cursor()

In [4]:
data = pd.read_csv("hospital.csv");
data
#df = pd.DataFrame(data, columns= ('Name','Cust_Id','Open_dt','Consul_dt','VACC_ID','Dr_Name','State','Country','DOB','FLAG'))
#type(df.Name)

,Customer_Name,Customer_Id,Open_dt,Consul_dt,Vaccination_Id,Dr_Name,State,Country,DOB,Is_Active
0,Alex,123457,2010-10-12,2012-10-13,MVD,Paul,SA,USA,1987-03-06,A
1,John,123458,2010-10-12,2012-10-13,MVD,Paul,TN,IND,1987-03-06,A
2,Mathew,123459,2010-10-12,2012-10-13,MVD,Paul,WAS,PHIL,1987-03-06,A
3,Matt,12345,2010-10-12,2012-10-13,MVD,Paul,BOS,NYC,1987-03-06,A
4,Jacob,1256,2010-10-12,2012-10-13,MVD,Paul,VIC,AU,1987-03-06,A
5,alex,123457,2010-10-12,2012-10-13,MVD,Paul,SA,USA,1987-03-06,A
6,marry,12347,2010-10-12,2012-10-13,MVD,Paul,SA,USA,1987-03-06,A


In [ ]:
sql ="INSERT INTO origin(Cust_name,Cust_I,Open_Dt,Consul_dt,VAC_ID,DR_Name,State,Country,DOB,FLAG) VALUES(%s,%s,%s,%s,%s,%s,%s,%s,%s,%s)"
 
for i, row in data.iterrows():
    col = (row.Customer_Name,str(row.Customer_Id),row.Open_dt,row.Consul_dt,row.Vaccination_Id,row.Dr_Name,row.State,row.Country,row.DOB,row.Is_Active)
    
   # print(col)
    mycursor.execute(sql,col)
   
    db.commit()
#mycursor.close()

# #Copy original table into staging area

In [ ]:
#mycursor.execute("create table MasterStage(Cust_name varchar(255) NOT NULL Primary key,Cust_I varchar(18) NOT NULL,Open_Dt DATE NOT NULL,Consul_Dt DATE,VAC_ID char(5),DR_Name char(255),State char(5),Country char(5),DOB DATE,FLAG char(1));")
#mycursor.execute("SELECT * INTO MasterStage FROM origin")
mycursor.execute("CREATE TABLE MasterStage AS SELECT * FROM origin")
db.commit()

In [6]:
#drop all tables except master and origin
mycursor.execute("drop table au,ind,nyc,phil,usa;")

In [7]:
mycursor.execute("show tables;")
tab=[]
#type(tables)
#mycursor.execute("SELECT * FROM Hospital.tables;")
tableslist = mycursor.fetchall()
for table in tableslist :
    tab.append(table)
    
print(tab)
tables = [''.join(i) for i in tab]
print(tables)

[('masterstage',), ('origin',)]
['masterstage', 'origin']


## Splitting Records into Different tables 

In [8]:
for i, row in data.iterrows():
  
    cnt = row.Country
    col = (row.Customer_Name,str(row.Customer_Id),row.Open_dt,row.Consul_dt,row.Vaccination_Id,row.Dr_Name,row.State,row.Country,row.DOB,row.Is_Active)
    
    try :
        if cnt in tables :
            
            mycursor.execute("INSERT INTO "+cnt +"(Cust_name,Cust_I,Open_Dt,Consul_dt,VAC_ID,DR_Name,State,Country,DOB,FLAG) VALUES(%s,%s,%s,%s,%s,%s,%s,%s,%s,%s)" ,col)
            
        else :
        
            mycursor.execute("CREATE TABLE "+ cnt +" (Cust_name varchar(255) NOT NULL Primary key,Cust_I varchar(18) NOT NULL,Open_Dt DATE NOT NULL,Consul_Dt DATE,VAC_ID char(5),DR_Name char(255),State char(5),Country char(5),DOB DATE,FLAG char(1));")
            mycursor.execute("INSERT INTO "+cnt +"(Cust_name,Cust_I,Open_Dt,Consul_dt,VAC_ID,DR_Name,State,Country,DOB,FLAG) VALUES(%s,%s,%s,%s,%s,%s,%s,%s,%s,%s)" ,col)
            tables.append(cnt)
    except:
        
        print("Duplicate entry with primary key")

    

db.commit()
db.close()

Duplicate entry with primary key
